In [7]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket.extensions.quantinuum import QuantinuumBackend, QuantinuumAPIOffline

api_offline = QuantinuumAPIOffline()
backend = QuantinuumBackend(device_name="H1-1LE", api_handler = api_offline)

circuit = Circuit()
qubits = circuit.add_q_register(name='qubits', size=3)
condition_reg = circuit.add_c_register(name='condition', size=1)
measure_reg = circuit.add_c_register(name='measure', size=2)

circuit.H(qubits[0])
circuit.Measure(qubit=qubits[0], bit=condition_reg[0])
circuit.X(qubit=qubits[1], condition=condition_reg[0])
circuit.Measure(qubit=qubits[1], bit=measure_reg[0])
circuit.Measure(qubit=qubits[2], bit=measure_reg[1])
render_circuit_jupyter(circuit)

backend.rebase_pass().apply(circuit)
result = backend.run_circuit(circuit, n_shots=100)
print("Measure register: ", result.get_counts(cbits=measure_reg))
print("Condition register: ", result.get_counts(cbits=condition_reg))

Measure register:  Counter({(0, 0): 51, (1, 0): 49})
Condition register:  Counter({(0,): 51, (1,): 49})


In [10]:
result.get_counts()

Counter({(0, 0, 0): 51, (1, 1, 0): 49})

In [ ]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket.extensions.quantinuum import QuantinuumBackend, QuantinuumAPIOffline

circ = Circuit()

# Initialise registers
q_register = circ.add_q_register(name='qubits', size=1)
c_register = circ.add_c_register(name='conditions', size=3)
m_register = circ.add_c_register(name='measures', size=1)

# Generate the random number
for cbit in c_register:
    circ.H(q_register[0])
    circ.Measure(q_register[0], cbit)
    circ.Reset(q_register[0])

# Add a barrier to seperate from main circuit
circ.add_barrier(q_register)

# Rotation
circ.Rz(0.25, q_register[0])

# Random measurement rotations
circ.T(q_register[0], condition=c_register[0])
circ.S(q_register[0], condition=c_register[1])
circ.Z(q_register[0], condition=c_register[2])

circ.Measure(q_register[0], m_register[0])

render_circuit_jupyter(circ)

api_offline = QuantinuumAPIOffline()
backend = QuantinuumBackend(device_name="H1-1LE", api_handler = api_offline)
compiled_circ = backend.get_compiled_circuit(circ, optimisation_level=0)
results = backend.run_circuit(compiled_circ, n_shots=1)

print(results.get_counts(cbits=m_register))
print(results.get_counts(cbits=c_register))